# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 28 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [ ]:
!pip install torchsummaryX==1.1.0 wandb --quiet
!pip install torch-optimizer


In [1]:
import torch
import datetime
import numpy as np
from torchsummaryX import summary
import sklearn
import random
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import torch_optimizer as optimx
from torch.cuda.amp import autocast, GradScaler
import os
import datetime
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
scaler = GradScaler()
print("Device: ", device)

Device:  cuda


In [ ]:
''' If you are using colab, you can import google drive to save model checkpoints in a folder
    If you want to use it, uncomment the two lines below
'''
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%pwd

'/content'

In [2]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [3]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"aaayush1","key":"9ad570c944c8b67cf242fb16c7cf3f40"}')
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

  Using cached kaggle-1.5.8-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
# commands to download data from kaggle
!kaggle competitions download -c 11785-hw1p2-f24

!unzip -qo /content/11785-hw1p2-f24.zip -d /content

100% 3.98G/3.98G [00:16<00:00, 264MB/s]
100% 3.98G/3.98G [00:16<00:00, 258MB/s]


# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [4]:
file_path = '/content/11785-f24-hw1p2/dev-clean/mfcc/1272-128104-0001.npy'

# Load the .npy file
data = np.load(file_path)

# Print the contents
print(data.shape)
print(data)


file_path = '/content/11785-f24-hw1p2/dev-clean/transcript/1272-128104-0001.npy'

# Load the .npy file
data = np.load(file_path)

# Print the contents
print(data.shape)

print(data)

(477, 28)
[[ 1.70536385e+01  1.83756161e+01  1.32918272e+01 ... -1.50571261e-02
  -6.27577156e-02  7.39348907e+01]
 [ 1.71380692e+01  1.84768028e+01  1.27026825e+01 ...  2.04224527e-01
  -1.66671127e-01  7.42002716e+01]
 [ 1.77663155e+01  1.70868053e+01  1.09276438e+01 ...  9.05482650e-01
  -9.12217647e-02  7.37023163e+01]
 ...
 [ 1.73955116e+01  4.50241947e+00  5.95822906e+00 ...  7.46107638e-01
   2.80462001e-02  7.80584183e+01]
 [ 1.65330410e+01  3.57640982e+00  4.13166904e+00 ... -5.02605319e-01
  -2.23344006e-02  7.97978287e+01]
 [ 1.45076504e+01  2.15292788e+00  6.49814081e+00 ... -4.83489990e-01
  -4.38078865e-02  8.09142075e+01]]
(479,)
['[SOS]' '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]'
 '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]'
 '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]'
 '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]'
 '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]' '[SIL]

In [ ]:
mfcc_dir = os.path.join('/content/11785-f24-hw1p2', 'train-clean-100', "mfcc")
# TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root

# TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
mfcc_names          = sorted(os.listdir(mfcc_dir))



# TODO: Iterate through mfccs and transcripts
mfcc_lengths = []

# Iterate through MFCC files
for i in range(len(mfcc_names)):
    mfcc_path = os.path.join(mfcc_dir, mfcc_names[i])

    # Load the MFCC
    mfcc = np.load(mfcc_path)

    # Append the number of time steps (shape[0]) to the list
    mfcc_lengths.append(mfcc.shape[0])

# Calculate the mean, min, and max
mean_length = np.mean(mfcc_lengths)
min_length = np.min(mfcc_lengths)
max_length = np.max(mfcc_lengths)

print(mean_length)
print(min_length)
print(max_length)

1264.6258453344547
137
2448


In [5]:
# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "train-clean-100", training = True): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes
        self.training   = training

        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir = os.path.join(root, partition, "mfcc")
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        self.transcript_dir = os.path.join(root, partition, "transcript")

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names          = sorted(os.listdir(self.mfcc_dir))
        # TODO: List files in self.transcript_dir using os.listdir in sorted order
        transcript_names    = sorted(os.listdir(self.transcript_dir))

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names)

        self.mfccs, self.transcripts = [], []

        # TODO: Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
            mfcc_path = os.path.join(self.mfcc_dir, mfcc_names[i])
            transcript_path = os.path.join(self.transcript_dir, transcript_names[i])
        #   Load a single mfcc
            mfcc        = np.load(mfcc_path)
        #   Do Cepstral Normalization of mfcc (explained in writeup)

            mfcc = mfcc - np.mean(mfcc, axis=0, keepdims=True)
            mfcc = mfcc / np.std(mfcc, axis=0, keepdims=True)

        #   Load the corresponding transcript
            transcript  = np.load(transcript_path)[1:-1] # Remove [SOS] and [EOS] from the transcript
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        # NOTE:
        # Each mfcc is of shape T1 x 28, T2 x 28, ...
        # Each transcript is of shape (T1+2), (T2+2) before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that
        # the final shape is T x 28 (Where T = T1 + T2 + ...)
        self.mfccs          = np.concatenate(self.mfccs, axis=0)

        # TODO: Concatenate all transcripts in self.transcripts such that
        # the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts    = np.concatenate(self.transcripts, axis=0)
        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        # Take some time to think about what we have done.
        # self.mfcc is an array of the format (Frames x Features).
        # Our goal is to recognize phonemes of each frame
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        padding = np.zeros((self.context, self.mfccs.shape[1]))
        self.mfccs = np.vstack([padding, self.mfccs, padding])

        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such.
        # Hence, we map these phonemes to integers

        # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        self.phoneme_to_int = {phoneme : i for i, phoneme in enumerate(self.phonemes)}
        self.transcripts = np.array([self.phoneme_to_int[phoneme] for phoneme in self.transcripts])
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def time_mask(self, mfcc, time_mask_fraction):
            """
            Apply time masking with a fraction of total frames.
            time_mask_fraction: Fraction of frames to mask (between 0 and 1).
            """
            t = mfcc.shape[0]
            max_time_mask = int(t * time_mask_fraction)
            if max_time_mask == 0:
                return mfcc

            # Choose a random mask length between 1 and max_time_mask
            time_mask_param = random.randint(0, max_time_mask)
            t0 = random.randint(0, t - time_mask_param)
            mfcc[t0:t0 + time_mask_param, :] = 0
            return mfcc


    def frequency_mask(self, mfcc, freq_mask_fraction):
        """
        Apply frequency masking with a fraction of total coefficients.
        freq_mask_fraction: Fraction of frequency bins to mask (between 0 and 1).
        """
        f = mfcc.shape[1]
        max_freq_mask = int(f * freq_mask_fraction)
        if max_freq_mask == 0:
            return mfcc

        # Choose a random mask length between 1 and max_freq_mask
        freq_mask_param = random.randint(0, max_freq_mask)
        f0 = random.randint(0, f - freq_mask_param)
        mfcc[:, f0:f0 + freq_mask_param] = 0
        return mfcc

    def apply_spec_augment(self, mfcc, time_mask_fraction, freq_mask_fraction):
        """
        Apply both time and frequency masking using fractions.
        """
        mfcc = self.time_mask(mfcc, time_mask_fraction)
        mfcc = self.frequency_mask(mfcc, freq_mask_fraction)
        return mfcc


    def __getitem__(self, ind):

        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        start_index = ind
        end_index = ind + 2 * self.context + 1
        frames = self.mfccs[start_index: end_index, :]

        if self.training:
          if random.random() <= 0.15:
            for i in range(2):
              frames = self.apply_spec_augment(frames, time_mask_fraction=0.04, freq_mask_fraction=0.05)


        # After slicing, you get an array of shape 2*context+1 x 28. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data

        frames      = torch.FloatTensor(frames) # Convert to tensors
        phonemes    = torch.tensor(self.transcripts[ind])

        return frames, phonemes

In [6]:
class AudioTestDataset(torch.utils.data.Dataset):
        def __init__(self, root, context=0, partition= "test-clean"): # Feel free to add more arguments

          self.context    = context
          self.mfcc_dir = os.path.join(root, partition, "mfcc")
          mfcc_names          = sorted(os.listdir(self.mfcc_dir))

          self.mfccs = []
          for i in range(len(mfcc_names)):
              mfcc_path = os.path.join(self.mfcc_dir, mfcc_names[i])
              mfcc        = np.load(mfcc_path)
              mfcc = mfcc - np.mean(mfcc, axis=0, keepdims=True)
              mfcc = mfcc / np.std(mfcc, axis=0, keepdims=True)
              self.mfccs.append(mfcc)

          self.mfccs          = np.concatenate(self.mfccs, axis=0)
          self.length = len(self.mfccs)
          padding = np.zeros((self.context, self.mfccs.shape[1]))
          self.mfccs = np.vstack([padding, self.mfccs, padding])

        def __len__(self):
          return self.length

        def __getitem__(self, ind):

            start_index = ind
            end_index = ind + 2 * self.context + 1
            frames = self.mfccs[start_index: end_index, :]
            frames = frames.flatten()
            frames      = torch.FloatTensor(frames)

            return frames



    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments.

In [7]:
config = {
    'epochs'        : 104,
    'batch_size'    : 2048,
    'context'       : 35,
    'init_lr'       : 1e-3,
    'architecture'  : '15th-iteration',
    'scheduler_type'    : 'CosineAnnealingRestart',
    'first_restart' : 4,
    'period_multipler' : 2,
    'decay_factor' : 0.7
}

# Create Datasets

In [8]:
root_dir = '/content/11785-f24-hw1p2'
#TODO: Create a dataset object using the AudioDataset class for the training data
train_data = AudioDataset(root=root_dir, partition="train-clean-100", context=config['context'], training = True)

# TODO: Create a dataset object using the AudioDataset class for the validation data
val_data = AudioDataset(root=root_dir, partition="dev-clean", context=config['context'], training = False)

# TODO: Create a dataset object using the AudioTestDataset class for the test data
test_data = AudioTestDataset(root=root_dir, partition= "test-clean", context=config['context'])

In [9]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 4,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*28)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  2048
Context        :  35
Input size     :  1988
Output symbols :  42
Train dataset samples = 36091157, batches = 17623
Validation dataset samples = 1928204, batches = 942
Test dataset samples = 1934138, batches = 945


In [10]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([2048, 1988]) torch.Size([2048])


In [ ]:
sil_count = 0
# Testing code to check if your data loaders are working
for i, data in enumerate(val_loader):
    frames, phoneme = data
    sil_count += (phoneme == 0).sum().item()

print("SIL in dev ", sil_count)

SIL in dev  319908


In [ ]:
from collections import defaultdict

phoneme_counts = defaultdict(int)

for i, data in enumerate(val_loader):
    frames, phoneme = data

    for p in phoneme:
        phoneme_counts[p.item()] += 1


for phoneme, count in phoneme_counts.items():
    print(f"Phoneme: {phoneme}, Count: {count}")


Phoneme: 0, Count: 319908
Phoneme: 22, Count: 44728
Phoneme: 17, Count: 74887
Phoneme: 29, Count: 101184
Phoneme: 31, Count: 97390
Phoneme: 12, Count: 54928
Phoneme: 20, Count: 47016
Phoneme: 36, Count: 37697
Phoneme: 21, Count: 65902
Phoneme: 38, Count: 54850
Phoneme: 10, Count: 37100
Phoneme: 3, Count: 123734
Phoneme: 27, Count: 34131
Phoneme: 1, Count: 29688
Phoneme: 35, Count: 27440
Phoneme: 9, Count: 62763
Phoneme: 2, Count: 49298
Phoneme: 23, Count: 94541
Phoneme: 18, Count: 70861
Phoneme: 28, Count: 62686
Phoneme: 15, Count: 13541
Phoneme: 34, Count: 26691
Phoneme: 11, Count: 47112
Phoneme: 16, Count: 34813
Phoneme: 4, Count: 29340
Phoneme: 24, Count: 19327
Phoneme: 14, Count: 37562
Phoneme: 37, Count: 9669
Phoneme: 25, Count: 30755
Phoneme: 7, Count: 23607
Phoneme: 6, Count: 49332
Phoneme: 13, Count: 36184
Phoneme: 5, Count: 20274
Phoneme: 32, Count: 9247
Phoneme: 8, Count: 12644
Phoneme: 30, Count: 17628
Phoneme: 19, Count: 8730
Phoneme: 26, Count: 3861
Phoneme: 39, Count: 869

In [ ]:
least_frequent_phoneme = min(phoneme_counts, key=phoneme_counts.get)
least_frequent_count = phoneme_counts[least_frequent_phoneme]
print(least_frequent_phoneme)

39


In [ ]:
print(val_data.phoneme_to_int)

{'[SIL]': 0, 'AA': 1, 'AE': 2, 'AH': 3, 'AO': 4, 'AW': 5, 'AY': 6, 'B': 7, 'CH': 8, 'D': 9, 'DH': 10, 'EH': 11, 'ER': 12, 'EY': 13, 'F': 14, 'G': 15, 'HH': 16, 'IH': 17, 'IY': 18, 'JH': 19, 'K': 20, 'L': 21, 'M': 22, 'N': 23, 'NG': 24, 'OW': 25, 'OY': 26, 'P': 27, 'R': 28, 'S': 29, 'SH': 30, 'T': 31, 'TH': 32, 'UH': 33, 'UW': 34, 'V': 35, 'W': 36, 'Y': 37, 'Z': 38, 'ZH': 39, '[SOS]': 40, '[EOS]': 41}


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [11]:
class Network(torch.nn.Module):

    def __init__(self, input_size, output_size):
        super(Network, self).__init__()

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 1995),
            torch.nn.BatchNorm1d(1995),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.17),

            #Hidden Layers - Start
            torch.nn.Linear(1995, 1995),
            torch.nn.BatchNorm1d(1995),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.20),

            torch.nn.Linear(1995, 1995),
            torch.nn.BatchNorm1d(1995),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.25),

            torch.nn.Linear(1995, 1995),
            torch.nn.BatchNorm1d(1995),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.20),

            torch.nn.Linear(1995, 1995),
            torch.nn.BatchNorm1d(1995),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.17),
            #Hidden Layers - End

            # Final output layer
            torch.nn.Linear(1995, output_size)
        )

    def forward(self, x):
        return self.model(x)

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler.

In [12]:
INPUT_SIZE  = (2*config['context'] + 1) * 28 # Why is this the case?
model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
summary(model, frames.to(device))
# Check number of parameters of your network
# Remember, you are limited to 20 million parameters for HW1 (including ensembles)

----------------------------------------------------------------------------------------------------
Layer                   Kernel Shape         Output Shape         # Params (K)      # Mult-Adds (M)
0_Linear                [1988, 1995]         [2048, 1995]             3,968.05                 3.97
1_BatchNorm1d                 [1995]         [2048, 1995]                 3.99                 0.00
2_ReLU                             -         [2048, 1995]                    -                    -
3_Dropout                          -         [2048, 1995]                    -                    -
4_Linear                [1995, 1995]         [2048, 1995]             3,982.02                 3.98
5_BatchNorm1d                 [1995]         [2048, 1995]                 3.99                 0.00
6_ReLU                             -         [2048, 1995]                    -                    -
7_Dropout                          -         [2048, 1995]                    -                    -

In [17]:
criterion = torch.nn.CrossEntropyLoss() # Defining Loss function.
# We use CE because the task is multi-class classification

# optimizer = torch.optim.AdamW(model.parameters(), lr=config['init_lr'], weight_decay=1e-4)

optimizer = torch.optim.Adam(model.parameters(), lr=config['init_lr'])


scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=7, T_mult=2)




# Recommended : Define Scheduler for Learning Rate,
# including but not limited to StepLR, MultiStep, CosineAnnealing, CosineAnnealingWithWarmRestarts, ReduceLROnPlateau, etc.
# You can refer to Pytorch documentation for more information on how to use them.

# Is your training time very high?
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [18]:
torch.cuda.empty_cache()
gc.collect()

873

In [19]:
def train(model, dataloader, optimizer, criterion):
    model.train()
    tloss, tacc = 0, 0  # Monitoring loss and accuracy
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, (frames, phonemes) in enumerate(dataloader):
        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        frames = frames.to(device)
        phonemes = phonemes.to(device)

        ### Mixed precision forward pass
        with autocast():
            logits = model(frames)
            loss = criterion(logits, phonemes)

        ### Backward pass and gradient scaling
        scaler.scale(loss).backward()

        ### Gradient Descent step (unscale before stepping optimizer)
        scaler.step(optimizer)

        ### Update the scaler
        scaler.update()

        tloss += loss.item()
        tacc += torch.sum(torch.argmax(logits, dim=1) == phonemes).item() / logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                              acc="{:.04f}%".format(float(tacc * 100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    tloss /= len(dataloader)
    tacc /= len(dataloader)

    return tloss, tacc

In [20]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb.

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [21]:
wandb.login(key="7664f3b17a98ffe7c64b549e349123b61a9d3024") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: Currently logged in as: aaayush (aaayush-carnegie-mellon-university). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [22]:
# Create your wandb run
run = wandb.init(
    name    = "sixteenth-run", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    #id     = "y28t31uz", ### Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account
    config  = config ### Wandb Config for your run
)

In [23]:
### Save your model architecture as a string with str(model)
model_arch  = str(model)

### Save it in a txt file
arch_file   = open("model_arch.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/content/wandb/run-20240912_142144-auxsi0j1/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [24]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
wandb.watch(model, log="all")

base_lr = config['init_lr']
decay_factor = config['decay_factor']
epochs = config['epochs']
first_restart = 6
second_restart = 12

for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc       = eval(model, val_loader)

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))


    ### Log metrics at each epoch in your run
    # Optionally, you can log at each batch inside train/eval functions
    # (explore wandb documentation/wandb recitation)
    wandb.log({
        'train_acc': train_acc * 100,
        'train_loss': train_loss,
        'val_acc': val_acc * 100,
        'valid_loss': val_loss,
        'lr': curr_lr,
    })


    scheduler.step()


    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best



Epoch 1/104


Train:   0%|          | 0/17623 [00:00<?, ?it/s]

Val:   0%|          | 0/942 [00:00<?, ?it/s]

	Train Acc 72.3760%	Train Loss 0.8566	 Learning Rate 0.0010000
	Val Acc 79.6073%	Val Loss 0.6109

Epoch 2/104


Train:   0%|          | 0/17623 [00:00<?, ?it/s]

Buffered data was truncated after reaching the output size limit.

In [25]:
# import os

checkpoint_dir = "checkpoints"

os.makedirs(checkpoint_dir, exist_ok=True)

model_save_path = os.path.join(checkpoint_dir, "model_epoch_16_1.pt")

torch.save(model.state_dict(), model_save_path)

wandb.save(model_save_path)

['/content/wandb/run-20240912_142144-auxsi0j1/files/checkpoints/model_epoch_16_1.pt']

# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
# model_save_path = os.path.join("checkpoints", "model_epoch_15_1.pt")

# model.load_state_dict(torch.load(model_save_path, map_location=torch.device('cuda')))


<All keys matched successfully>

In [26]:
int_to_phoneme = {i: phoneme for phoneme, i in train_data.phoneme_to_int.items()}

def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval()# TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.no_grad(): # TODO

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)

            logits  = model(mfccs)

            predicted_phoneme_ids = torch.argmax(logits, dim=1)

            predicted_phonemes = [int_to_phoneme[id.item()] for id in predicted_phoneme_ids]

            test_predictions.extend(predicted_phonemes)

    return test_predictions

In [27]:
predictions = test(model, test_loader)

  0%|          | 0/945 [00:00<?, ?it/s]

In [28]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [ ]:
### Finish your wandb run
run.finish()

In [29]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
!kaggle competitions submit -c 11785-hw1p2-f24 -f ./submission.csv -m "Test Submission"

### However, its always safer to download the csv file and then upload to kaggle

100% 19.3M/19.3M [00:01<00:00, 18.6MB/s]
Successfully submitted to 11785 HW1P2 Fall 2024